<a href="https://colab.research.google.com/github/Shashank-u803/BIS-Lab/blob/main/Week%205/Cuckoo_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import random
import math

# Step 1: Define the Problem (Optimization Function)
def sphere_function(x):
    """Simple sphere function: f(x) = sum(x_i^2)
    Global minimum at x = [0, 0, ...] with f(x) = 0"""
    return sum(xi**2 for xi in x)

# Step 5: Generate Lévy Flight
def levy_flight(dimension):
    """Generate step size using Lévy flight distribution"""
    beta = 1.5
    sigma_u = (math.gamma(1 + beta) * math.sin(math.pi * beta / 2) /
               (math.gamma((1 + beta) / 2) * beta * 2**((beta - 1) / 2)))**(1 / beta)

    u = random.gauss(0, sigma_u)
    v = random.gauss(0, 1)
    step = u / abs(v)**(1 / beta)

    return [step * random.gauss(0, 1) for _ in range(dimension)]

# Main Cuckoo Search Algorithm
def cuckoo_search(objective_func, dim=2, n_nests=25, pa=0.25, max_iter=100, bounds=(-10, 10)):
    """
    Cuckoo Search Optimization Algorithm

    Parameters:
    - objective_func: function to minimize
    - dim: dimension of the problem
    - n_nests: number of nests (population size)
    - pa: probability of discovery (fraction of worst nests to abandon)
    - max_iter: maximum number of iterations
    - bounds: search space bounds (min, max)
    """

    # Step 3: Initialize Population (nests with random positions)
    nests = [[random.uniform(bounds[0], bounds[1]) for _ in range(dim)]
             for _ in range(n_nests)]

    # Step 4: Evaluate Fitness
    fitness = [objective_func(nest) for nest in nests]

    # Track the best solution
    best_nest_idx = fitness.index(min(fitness))
    best_nest = nests[best_nest_idx][:]
    best_fitness = fitness[best_nest_idx]

    print(f"Initial Best Solution: [{', '.join(f'{x:.6f}' for x in best_nest)}]")
    print(f"Initial Best Fitness: f(x) = {best_fitness:.6f}\n")

    # Step 7: Iterate
    for iteration in range(max_iter):

        # Step 5: Generate New Solutions via Lévy Flights
        for i in range(n_nests):
            # Get a cuckoo randomly by Lévy flights
            step_size = levy_flight(dim)
            new_nest = [nests[i][j] + step_size[j] for j in range(dim)]

            # Apply bounds
            new_nest = [max(bounds[0], min(bounds[1], x)) for x in new_nest]

            # Evaluate new solution
            new_fitness = objective_func(new_nest)

            # Choose a random nest (not the same one)
            j = random.randint(0, n_nests - 1)

            # Replace if the new solution is better
            if new_fitness < fitness[j]:
                nests[j] = new_nest
                fitness[j] = new_fitness

        # Step 6: Abandon Worst Nests
        # Sort nests by fitness to identify worst ones
        sorted_indices = sorted(range(n_nests), key=lambda k: fitness[k], reverse=True)
        n_abandon = int(pa * n_nests)

        for i in range(n_abandon):
            idx = sorted_indices[i]
            # Replace with new random position
            nests[idx] = [random.uniform(bounds[0], bounds[1]) for _ in range(dim)]
            fitness[idx] = objective_func(nests[idx])

        # Update best solution
        current_best_idx = fitness.index(min(fitness))
        if fitness[current_best_idx] < best_fitness:
            best_fitness = fitness[current_best_idx]
            best_nest = nests[current_best_idx][:]

        # Print progress every 20 iterations
        if (iteration + 1) % 20 == 0:
            print(f"Iteration {iteration + 1:3d}: x = [{', '.join(f'{x:.6f}' for x in best_nest)}] | f(x) = {best_fitness:.6f}")

    # Step 8: Output the Best Solution
    return best_nest, best_fitness

# Run the algorithm
if __name__ == "__main__":
    print("=" * 60)
    print("        Cuckoo Search Optimization Algorithm")
    print("=" * 60)
    print(f"Function Name    : sphere_function(x)")
    print(f"Mathematical Form: f(x) = Σ(x_i²)")
    print(f"Dimensions       : 2")
    print(f"Search Bounds    : [-10, 10]")
    print(f"Global Minimum   : f(0, 0) = 0")
    print("=" * 60)
    print()

    # Run Cuckoo Search
    best_solution, best_value = cuckoo_search(
        objective_func=sphere_function,
        dim=2,
        n_nests=25,
        pa=0.25,
        max_iter=100,
        bounds=(-10, 10)
    )

    print("\n" + "=" * 60)
    print("                        RESULTS")
    print("=" * 60)
    print(f"Optimal Solution : x = [{', '.join(f'{x:.6f}' for x in best_solution)}]")
    print(f"Optimal Value    : f(x) = {best_value:.6f}")
    print(f"Function         : sphere_function(x) = Σ(x_i²)")
    print("=" * 60)

        Cuckoo Search Optimization Algorithm
Function Name    : sphere_function(x)
Mathematical Form: f(x) = Σ(x_i²)
Dimensions       : 2
Search Bounds    : [-10, 10]
Global Minimum   : f(0, 0) = 0

Initial Best Solution: [-1.618897, 1.504998]
Initial Best Fitness: f(x) = 4.885847

Iteration  20: x = [0.000180, 0.013211] | f(x) = 0.000175
Iteration  40: x = [0.003584, -0.003925] | f(x) = 0.000028
Iteration  60: x = [-0.003263, -0.003471] | f(x) = 0.000023
Iteration  80: x = [-0.002295, -0.001751] | f(x) = 0.000008
Iteration 100: x = [-0.002295, -0.001751] | f(x) = 0.000008

                        RESULTS
Optimal Solution : x = [-0.002295, -0.001751]
Optimal Value    : f(x) = 0.000008
Function         : sphere_function(x) = Σ(x_i²)
